### Restaurant Order Analysis

Analyze order data to identify the most and least popular menu items and types of cuisine

#### Problem Statement:

You've just been hired as a Data Analyst for the Taste of the World Cafe, a restaurant that has diverse menu offerings and serves generous portions. The Taste of the World Cafe debuted a new menu at the start of the year. You've been asked to dig into the customer data to see which menu items are doing well / not well and what the top customers seem to like the most.

#### Objectives: 
<b>Objective 1: </b>Explore the items table

Your first objective is to better understand the items table by finding the number of rows in the table, the least and most expensive items, and the item prices within each category.

<b>Objective 2: </b>Explore the orders table

Your second objective is to better understand the orders table by finding the date range, the number of items within each order, and the orders with the highest number of items.

<b>Objective 3: </b>Analyze customer behavior

Your final objective is to combine the items and orders tables, find the least and most ordered categories, and dive into the details of the highest spend orders.

In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import mysql.connector
from datetime import datetime

In [92]:
# conn = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     password="password",
#     database="restaurant_db"
# )

# query1 = "SELECT * FROM menu_items"
# query2 = "SELECT * FROM order_details"
# menu_items = pd.read_sql(query1, conn)
# order_details = pd.read_sql(query2, conn)

# conn.close()

In [93]:
# menu_items.to_csv("menu_items.csv", index=False)
# order_details.to_csv("order_details.csv", index=False)

In [94]:
# read the csv datasets
menu_items = pd.read_csv("menu_items.csv")
order_details = pd.read_csv("order_details.csv")

In [95]:
print(menu_items.shape)
print(order_details.shape)

(32, 4)
(12234, 5)


In [96]:
menu_items.head()

,menu_item_id,item_name,category,price
0,101,Hamburger,American,12.95
1,102,Cheeseburger,American,13.95
2,103,Hot Dog,American,9.00
3,104,Veggie Burger,American,10.50
4,105,Mac & Cheese,American,7.00


In [97]:
menu_items.dtypes

menu_item_id      int64
item_name        object
category         object
price           float64
dtype: object

In [98]:
order_details.head()

,order_details_id,order_id,order_date,order_time,item_id
0,1,1,2023-01-01,0 days 11:38:36,109.0
1,2,2,2023-01-01,0 days 11:57:40,108.0
2,3,2,2023-01-01,0 days 11:57:40,124.0
3,4,2,2023-01-01,0 days 11:57:40,117.0
4,5,2,2023-01-01,0 days 11:57:40,129.0


In [99]:
order_details.dtypes

order_details_id      int64
order_id              int64
order_date           object
order_time           object
item_id             float64
dtype: object

In [100]:
order_details['item_id'] = order_details['item_id'].convert_dtypes('int')
order_details['order_date'] = pd.to_datetime(order_details['order_date'])
order_details['order_time'] = order_details['order_time'].str.split().str[-1]
order_details['order_time'] = order_details['order_time'].apply(lambda x: datetime.strptime(x, "%H:%M:%S").time())
order_details["hour"] = order_details['order_time'].apply(lambda x: x.hour)

In [101]:
order_details.head()

,order_details_id,order_id,order_date,order_time,item_id,hour
0,1,1,2023-01-01,11:38:36,109,11
1,2,2,2023-01-01,11:57:40,108,11
2,3,2,2023-01-01,11:57:40,124,11
3,4,2,2023-01-01,11:57:40,117,11
4,5,2,2023-01-01,11:57:40,129,11
